In [1]:
import os
import time
import random
import collections

import numpy as np
import tensorflow as tf

In [2]:
tf.__version__

'2.8.0'

In [3]:
# ট্রেনিং প্যারামিটার
LEARNING_RATE = 0.1
BATCH_SIZE = 2048
NUM_STEPS = 400000
DISPLAY_STEPS = 10000
EVAL_STEPS = 20000

# এই শব্দগুলো দিয়ে আমরা মডেল ট্রেনিং করার সময় টেস্ট করে দেখব যে মডেল কেমন শিখতেছে
EVAL_WORDS = ['মিউটেশন','খতিয়ানের','নামজারি','পর্চা','মৌজা']

# এম্বেডিং ভেক্টরে ডাইমেসন। একটা শব্দের বিপরীতে ২০০ টা স্কেলার ভেলু জমা থাকবে
# এবং এটাই হবে ওই শব্দের ভেক্টর রিপ্রেজেন্টেসন
EMBEDDING_SIZE = 200

# আমাদের করপাস থেকে এর থেকে বেশি শব্দ থাকলে সেগুলো কে বাদ দেব ।
MAX_VOCABULARY_SIZE = 50000

# যে শব্দগুলো আমাদের ডাটা তে খুব কম সংখ্যক বার আছে সেগুলো আমারা বাদ দিব ।
MIN_OCCURRENCE = 10

# যখন আমরা ডাটা বানাব তখন আমাদের লেফট এবং রাইট সাইট এর কতগুলো শব্দও ব্যাবহার করব
SKIP_WINDOW = 3

# আমরা একই শব্দ গিয়ে কত গুলো ডাটা বানাতে চাই তার সংখ্যা
NUM_SKIPS = 2

# লস হিসাব করার সময় কতগুলো নেগেটিভ ডাটা ব্যাবহার করতে চাই
NUM_SAMPLED = 64

In [4]:
corpus = '/home/word2vector/data.txt'

unique_chars = set()
with open(corpus, 'r') as fp:
    lines = fp.readlines()
    # find unique chars so that we can remove unwanted chars
    for line in lines:
        line = line.strip()
        if line:
            for c in line:
                unique_chars.add(c)
    unique_chars = sorted(unique_chars)
    print("Unique chars:", unique_chars)

Unique chars: [' ', 'x', 'ঁ', 'ং', 'ঃ', 'অ', 'আ', 'ই', 'উ', 'ঊ', 'এ', 'ঐ', 'ও', 'ক', 'খ', 'গ', 'ঘ', 'ঙ', 'চ', 'ছ', 'জ', 'ঝ', 'ঞ', 'ট', 'ঠ', 'ড', 'ঢ', 'ণ', 'ত', 'থ', 'দ', 'ধ', 'ন', 'প', 'ফ', 'ব', 'ভ', 'ম', 'য', 'র', 'ল', 'শ', 'ষ', 'স', 'হ', '়', 'া', 'ি', 'ী', 'ু', 'ূ', 'ৃ', 'ে', 'ৈ', 'ো', 'ৌ', '্', 'ৎ', 'ৗ', 'ড়', 'য়', '\u200d']


In [5]:
def clean(filter_list, text):
    for ch in filter_list:
        text = text.replace(ch, '')
    text = text.strip()
    return text

# for dataset version 1.2
chars_to_remove = [
    '!', '(', ')', ',', '-', '.', '1', ':', ';', '<', '?', 'R', 'r', '{', '·',
    'я', 'अ', 'छ', 'ब', 'भ', 'य', 'र', 'ल', 'श', 'ৰ', 'ৱ', '৷', '–', '—',
    '‘', '’', '“', '”', '…', '।'
]

text_words = []
for line in lines:
    text = clean(chars_to_remove, line)
    if text and text != '':
        words = text.split()
        text_words.extend(words)

print("Total words:", len(text_words))

Total words: 14450


In [6]:
# Unknown ওয়ার্ড এর জন্য UNK টকেন ব্যাবহার করব ।
dictionary = [('UNK', -1)]
# MAX_VOCABULARY_SIZE ধরে কমন (মানে যে শব্দও গুলোর ফ্রিকুয়েন্সি বেশি) সেগুলো নিয়ে নিলাম
dictionary.extend(
    collections.Counter(text_words).most_common(MAX_VOCABULARY_SIZE - 1)
)

print("Total unique words:", len(dictionary))
print("Dictionary of unique words", dictionary)

Total unique words: 2355
Dictionary of unique words [('UNK', -1), ('কি', 357), ('ভূমি', 324), ('অনলাইনে', 307), ('না', 239), ('খতিয়ান', 167), ('বা', 165), ('করতে', 165), ('কর', 163), ('আবেদন', 163), ('করে', 160), ('পর্চা', 151), ('নামজারি', 151), ('করা', 147), ('উন্নয়ন', 139), ('অনলাইন', 135), ('আমার', 130), ('কিভাবে', 125), ('আমি', 123), ('হবে', 112), ('জন্য', 110), ('জমি', 98), ('যাবে', 94), ('জমির', 92), ('আছে', 89), ('থেকে', 86), ('খাজনা', 85), ('কোনো', 84), ('করুন', 74), ('করার', 74), ('ও', 73), ('এই', 72), ('ডট', 65), ('মৌজা', 64), ('এখন', 62), ('দুই', 61), ('কপি', 57), ('জানতে', 56), ('এক', 56), ('যায়', 55), ('হয়', 55), ('অফিসে', 53), ('মাধ্যমে', 52), ('কিন্তু', 52), ('দাগ', 52), ('দিতে', 51), ('বিস্তারিত', 50), ('ম্যাপ', 48), ('টু', 47), ('স্ল্যাশ', 47), ('এবং', 45), ('ওয়ান', 45), ('দলিল', 44), ('করবো', 43), ('হোল্ডিং', 42), ('দেওয়া', 42), ('সেবা', 41), ('নামজারির', 41), ('আপনার', 40), ('পরিশোধ', 40), ('তথ্য', 39), ('পাওয়া', 39), ('দিয়ে', 39), ('আমাদের', 38), ('টাকা', 38), ('পে

In [7]:
# Remove samples with less than 'min_occurrence' occurrences.
# যে শব্দ গুলো 'MIN_OCCURRENCE' থেকে ছোট সেগুলোকে আমরা বাদ দিয়ে দেই
for i in range(len(dictionary)-1, -1, -1):
    # i'th শব্দ কতবার এসেছে সেটা নেই
    occurrence = dictionary[i][1]
    # যদি MIN_OCCURRENCE থেকে ছোট হয় তাহলে আমরা এই শব্দকে dictionary থেক বাদ দেই
    if occurrence < MIN_OCCURRENCE:
        dictionary.pop(i)
    else:
        # যেহেতু আমাদের dictionary টা occurrence এর উপর ভিত্তি করে সর্ট (বেশি থেকে কম) করা
        # এবং আমরা এটাকে শেষ থেকে শুরুর দিকে ইতারেট করতেছি এই জন্য আমরা যদি কোন শব্দের জন্য
        # occurrence < MIN_OCCURRENCE না পাই তাহলে আমরা লুপ থেকে বের হয়ে যাব। কারণ সবগুলো
        # অলরেডি বাদ পরে গেছে ।
        break

VOCABULARY_SIZE = len(dictionary)
print("Vocabulary size after filter min occurrence word:", VOCABULARY_SIZE)

Vocabulary size after filter min occurrence word: 281


In [8]:
word2id = {}
id2word = {}
for i, (word, _)in enumerate(dictionary):
    # প্রত্যেক টা শব্দকে আমরা একটা আইডি এসাইন করে দেই
    word2id[word] = i
    # আইডি থেকে আবার ওয়ার্ড এ ফিরে আসার জন্য ট্রাক রাখি
    id2word[i] = word
print('word2id:', word2id)
print('id2word:', id2word)

word2id: {'UNK': 0, 'কি': 1, 'ভূমি': 2, 'অনলাইনে': 3, 'না': 4, 'খতিয়ান': 5, 'বা': 6, 'করতে': 7, 'কর': 8, 'আবেদন': 9, 'করে': 10, 'পর্চা': 11, 'নামজারি': 12, 'করা': 13, 'উন্নয়ন': 14, 'অনলাইন': 15, 'আমার': 16, 'কিভাবে': 17, 'আমি': 18, 'হবে': 19, 'জন্য': 20, 'জমি': 21, 'যাবে': 22, 'জমির': 23, 'আছে': 24, 'থেকে': 25, 'খাজনা': 26, 'কোনো': 27, 'করুন': 28, 'করার': 29, 'ও': 30, 'এই': 31, 'ডট': 32, 'মৌজা': 33, 'এখন': 34, 'দুই': 35, 'কপি': 36, 'জানতে': 37, 'এক': 38, 'যায়': 39, 'হয়': 40, 'অফিসে': 41, 'মাধ্যমে': 42, 'কিন্তু': 43, 'দাগ': 44, 'দিতে': 45, 'বিস্তারিত': 46, 'ম্যাপ': 47, 'টু': 48, 'স্ল্যাশ': 49, 'এবং': 50, 'ওয়ান': 51, 'দলিল': 52, 'করবো': 53, 'হোল্ডিং': 54, 'দেওয়া': 55, 'সেবা': 56, 'নামজারির': 57, 'আপনার': 58, 'পরিশোধ': 59, 'তথ্য': 60, 'পাওয়া': 61, 'দিয়ে': 62, 'আমাদের': 63, 'টাকা': 64, 'পেমেন্ট': 65, 'পাবো': 66, 'কল': 67, 'হয়েছে': 68, 'করলে': 69, 'করেছি': 70, 'নম্বর': 71, 'অফিস': 72, 'নাগরিক': 73, 'যদি': 74, 'রেকর্ড': 75, 'জমা': 76, 'কেন': 77, 'নং': 78, 'হলে': 79, 'নাম্বারে': 80, 'হেল্পলাই

In [9]:
data = []
unk_word_count = 0
for word in text_words:
    # Retrieve a word id, or assign it index 0 ('UNK') if not in dictionary.
    # text_words এর প্রতিটি শব্দকে একটা আইডি দিয়ে দেই ।
    # যদি শব্দটা টা word2id মধ্যে পাওয়া না যায় তাহলে এটা একটা UNK ক্যাটাগরির শব্দ
    # এই জন্য এটাকে আমরা ০ আইডি দিয়ে দিব
    index = word2id.get(word, 0)

    # UNK টকেনের ফ্রিকুয়েন্সি কেমন হয় সেটা ট্রাক রাখা
    if index == 0:
        unk_word_count += 1
    # প্রতিটা শব্দের বিপরীতে আমরা সেই শব্দের আইডি কে রাখলাম এবং এই ডাটা দিয়েই আমরা আমাদের মডেল
    # ট্রেনিং করব ।
    data.append(index)
# UNK এর ফ্রিকুয়েন্সি ডিকশনারিতে এড করে দিলাম
dictionary[0] = ('UNK', unk_word_count)

# শব্দকে আইডি দেয়ার পরের অবস্থা
print("data        :", data)
print("Totla data  :", len(data))
print('-' * 50)
# আইডি থেকে আবার শব্দে ফিরে আসার পরের অবস্থা
print("data in text:", [id2word[id] for id in data])
print('-' * 50)


print("Words count:", len(text_words))
print("Unique words:", len(set(text_words)))
print("Vocabulary size:", VOCABULARY_SIZE)
print("Most common words:", dictionary[:20])

data        : [50, 112, 0, 23, 166, 0, 0, 23, 0, 0, 0, 10, 23, 166, 0, 167, 126, 105, 0, 23, 166, 25, 0, 0, 35, 120, 0, 202, 0, 203, 42, 16, 0, 255, 25, 0, 0, 21, 0, 0, 0, 0, 256, 0, 0, 121, 0, 0, 0, 8, 24, 1, 4, 85, 235, 17, 37, 87, 0, 0, 0, 0, 190, 0, 79, 0, 88, 2, 41, 176, 28, 204, 122, 46, 37, 2, 56, 81, 51, 98, 51, 48, 48, 236, 67, 28, 74, 0, 0, 257, 237, 0, 0, 205, 23, 89, 0, 88, 2, 72, 25, 0, 10, 3, 11, 0, 10, 23, 0, 37, 19, 153, 206, 47, 50, 5, 0, 0, 0, 0, 75, 0, 0, 0, 203, 0, 3, 2, 14, 8, 207, 13, 106, 191, 94, 94, 192, 177, 32, 138, 21, 0, 223, 35, 0, 86, 258, 178, 4, 31, 0, 0, 0, 193, 25, 63, 0, 10, 206, 224, 206, 224, 259, 28, 58, 90, 0, 3, 225, 13, 0, 226, 107, 127, 0, 20, 128, 0, 41, 6, 3, 9, 7, 133, 154, 25, 58, 0, 20, 0, 168, 155, 205, 0, 23, 237, 0, 0, 0, 227, 79, 6, 0, 0, 0, 16, 86, 43, 0, 95, 75, 82, 0, 91, 70, 57, 0, 30, 0, 0, 23, 237, 16, 15, 2, 14, 0, 126, 156, 24, 208, 0, 24, 121, 134, 204, 122, 46, 37, 2, 56, 81, 51, 98, 51, 48, 48, 80, 67, 28, 74, 99, 139, 0, 4

In [10]:
data_index = 0
# Generate training batch for the skip-gram model.
def next_batch(batch_size, num_skips, skip_window):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    # get window size (words left and right + current one).
    span = 2 * skip_window + 1
    # difine a deque buffer to store span words
    # buffer words will change over iteration
    buffer = collections.deque(maxlen=span)
    if data_index + span > len(data):
        data_index = 0
    # select span of words
    buffer.extend(data[data_index:data_index + span])
    data_index += span
    # prepare batch data
    for i in range(batch_size // num_skips):
        # for span = 7 and skip_window = 3
        # span range = [0, 1, 2, 3, 4, 5, 6]
        # 3 will be discard
        # context_word = [0, 1, 2, 4, 5, 6]
        context_words = [w for w in range(span) if w != skip_window]
        # randomly take `num_skips` sample from context_word
        words_to_use = random.sample(context_words, num_skips)
        for j, context_word in enumerate(words_to_use):
            # j = 0 মুনা, j = 1 মুন
            batch[i * num_skips + j] = buffer[skip_window]
            # j = 0 গেটের, j = 1 এসে
            labels[i * num_skips + j, 0] = buffer[context_word]
        if data_index == len(data):
            # if we reach end to take '0 to span' words
            buffer.extend(data[0:span])
            # update data index
            data_index = span
        else:
            # update buffer by new word
            buffer.append(data[data_index])
            data_index += 1
    # শেষের শব্দগুলো যাতে বাদ না পরে এই জন্য ইনডেক্স টা আপডেট করে দিলাম
    data_index = (data_index + len(data) - span) % len(data)
    return batch, labels

In [11]:
batch_x, batch_y = next_batch(BATCH_SIZE, NUM_SKIPS, SKIP_WINDOW)
print("shape target:", batch_x.shape)
print('shape context:', batch_y.shape)
print('-' * 50)

print('target word :', end='')
for i in range(len(batch_x)):
    print(id2word[batch_x[i]], end=',')
print('')
print('-' * 50)

print("context word:", end='')
for i in range(len(batch_x)):
    print(id2word[batch_y[i][0]], end=',')


shape target: (2048,)
shape context: (2048, 1)
--------------------------------------------------
target word :জমির,জমির,পরিমাণ,পরিমাণ,UNK,UNK,UNK,UNK,জমির,জমির,UNK,UNK,UNK,UNK,UNK,UNK,করে,করে,জমির,জমির,পরিমাণ,পরিমাণ,UNK,UNK,আবার,আবার,একটা,একটা,দাগে,দাগে,UNK,UNK,জমির,জমির,পরিমাণ,পরিমাণ,থেকে,থেকে,UNK,UNK,UNK,UNK,দুই,দুই,হাজার,হাজার,UNK,UNK,সালে,সালে,UNK,UNK,দলিলের,দলিলের,মাধ্যমে,মাধ্যমে,আমার,আমার,UNK,UNK,কাছ,কাছ,থেকে,থেকে,UNK,UNK,UNK,UNK,জমি,জমি,UNK,UNK,UNK,UNK,UNK,UNK,UNK,UNK,বর্তমানে,বর্তমানে,UNK,UNK,UNK,UNK,দাখিলা,দাখিলা,UNK,UNK,UNK,UNK,UNK,UNK,কর,কর,আছে,আছে,কি,কি,না,না,তা,তা,সঠিকভাবে,সঠিকভাবে,কিভাবে,কিভাবে,জানতে,জানতে,পারবো,পারবো,UNK,UNK,UNK,UNK,UNK,UNK,UNK,UNK,জানাবেন,জানাবেন,UNK,UNK,হলে,হলে,UNK,UNK,ইউনিয়ন,ইউনিয়ন,ভূমি,ভূমি,অফিসে,অফিসে,যোগাযোগ,যোগাযোগ,করুন,করুন,বিষয়টি,বিষয়টি,সম্পর্কে,সম্পর্কে,বিস্তারিত,বিস্তারিত,জানতে,জানতে,ভূমি,ভূমি,সেবা,সেবা,হেল্পলাইন,হেল্পলাইন,ওয়ান,ওয়ান,সিক্স,সিক্স,ওয়ান,ওয়ান,টু,টু,টু,টু,নম্বরে,নম্বরে,কল,কল,করুন,করুন,যদি,যদি,UNK,UNK,UNK,UNK,সূত্রে,সূত্রে,মালিকানা,

In [12]:

# এই ভেরিয়েবলে আমরা আমাদের এম্বিডিং কে স্টোর করে রাখবে
embedding = tf.Variable(tf.random.normal([VOCABULARY_SIZE, EMBEDDING_SIZE]))

# এই ভেরিয়েবল গুলো NCE loss হিসাব করার সময় ব্যাবহার হবে
nce_weights = tf.Variable(tf.random.normal([VOCABULARY_SIZE, EMBEDDING_SIZE]))
nce_biases = tf.Variable(tf.zeros([VOCABULARY_SIZE]))

2023-12-12 11:52:27.254498: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10794 MB memory:  -> device: 0, name: Tesla K80, pci bus id: 0000:00:09.0, compute capability: 3.7


In [13]:
def get_embedding(word):
    # প্রতিটা ওয়ার্ড এর বিপরীতে 'embedding' ভেরিয়েবলে যে embedding স্টোর করা আছে সেটা বের করা
    x_embed = tf.nn.embedding_lookup(embedding, word)
    return x_embed

def nce_loss(x_embed, y):
    # একটা ব্যাচ ডাটার বিপরীতে NCE loss হিসাব করা
    y = tf.cast(y, tf.int64)
    loss = tf.reduce_mean(
        tf.nn.nce_loss(
            weights=nce_weights,
            biases=nce_biases,
            labels=y,
            inputs=x_embed,
            num_sampled=NUM_SAMPLED, # number of negative sample used
            num_classes=VOCABULARY_SIZE
        )
    )
    return loss


def cosing_similarity(x_embed):
    # ইনপুট এম্বেডিং ভেক্টরের সাথে প্রতিটা এম্বেডিং ভেক্টরের cosine similarity বের করা
    x_embed = tf.cast(x_embed, tf.float32)
    x_embed_norm = x_embed / tf.sqrt(tf.reduce_sum(tf.square(x_embed)))
    embedding_norm = embedding / tf.sqrt(
        tf.reduce_sum(tf.square(embedding), 1, keepdims=True), tf.float32
    )
    cosine_sim_op = tf.matmul(x_embed_norm, embedding_norm, transpose_b=True)
    return cosine_sim_op

# আমরা SGD optimizer ব্যাবহার করব
optimizer = tf.optimizers.SGD(LEARNING_RATE)

In [ ]:
x_test = np.array([word2id[w] for w in EVAL_WORDS])

start_time = time.time()
for step in range(1, NUM_STEPS+1):
    # ট্রেনিং এর জন্য ডাটা নেই
    target_x, context_y = next_batch(BATCH_SIZE, NUM_SKIPS, SKIP_WINDOW)

    with tf.GradientTape() as tape:
        # টার্গেট শব্দগুলোর বর্তমান এম্বেডিং বের করি
        emb = get_embedding(target_x)
        # এম্বিডিং এবং কন্টেক্সট থেকে লস হিসাব করি ।
        loss = nce_loss(emb, context_y)
    # loss এর সাপেক্ষে embedding, nce_weights, nce_biases ভেরিয়েবল গুলোর গ্রাডিয়েন্ট হিসাব করা
    gradients = tape.gradient(loss, [embedding, nce_weights, nce_biases])
    # এই গ্রাডিয়েন্ট ধরে আমরা embedding, nce_weights, nce_biases ভেরিয়েবল গুলোর ভেলু আপডেট
    # করি
    optimizer.apply_gradients(zip(gradients, [embedding, nce_weights, nce_biases]))

    # নিদিষ্ট স্টেপ পর পর আমরা লস দেখব
    if step % DISPLAY_STEPS == 0 or step == 1:
        temp_emb = get_embedding(batch_x)
        loss = nce_loss(temp_emb, batch_y)
        print("Step: {} loss: {:.4f} time: {}".format(
            step, loss, time.time()-start_time)
        )
        start_time = time.time()

    # আমাদের সেট করা কিছু টেস্ট শব্দ দিয়ে আমরা টেস্ট করে দেখব আমাদের মডেল কেমন শিখতেছে
    if step % EVAL_STEPS == 0 or step == 1:
        print("Testing...")
        similarity = cosing_similarity(get_embedding(x_test)).numpy()
        for i in range(len(EVAL_WORDS)):
            top_k = 8  # আমরা কতগুলো নেয়ারেস্ট শব্দ দেখতে চাই সেটা সেট করে দিলাম
            nearest = (-similarity[i, :]).argsort()[1:top_k+1]
            log_str = "'{}' এর কাছের শব্দ গুলো: ".format(EVAL_WORDS[i])
            for k in range(top_k):
                log_str = "{} {},".format(log_str, id2word[nearest[k]])
            print(log_str)


Step: 1 loss: 379.6901 time: 0.4335651397705078
Testing...
'মিউটেশন' এর কাছের শব্দ গুলো:  ম্যাপ, কাগজপত্র, ভূমি, লাগবে, কেন, যাবে, জানা, ব্যবহার,
'খতিয়ানের' এর কাছের শব্দ গুলো:  আপিল, নামে, করলে, আইন, হোল্ডিং, জি, দিন, যদি,
'নামজারি' এর কাছের শব্দ গুলো:  হবে, দুই, এবং, খতিয়ানে, দিতে, জেলার, এখন, বর্তমানে,
'পর্চা' এর কাছের শব্দ গুলো:  স্ল্যাশ, হওয়ার, থাকলে, দিয়ে, অ্যাঁ, একই, ইউনিয়ন, সেটা,
'মৌজা' এর কাছের শব্দ গুলো:  অপশনে, যার, আছে, মালিকের, জমা, নাম, জানাবেন, ফি,
Step: 10000 loss: 4.8408 time: 504.2725021839142
Step: 20000 loss: 4.3586 time: 500.05004143714905
Testing...
'মিউটেশন' এর কাছের শব্দ গুলো:  ম্যাপ, লাগবে, কাগজপত্র, জেলা, এবং, জানা, যাবে, দুই,
'খতিয়ানের' এর কাছের শব্দ গুলো:  নামে, ভূমি, যদি, আপিল, করলে, ঠিক, জি, অনলাইনের,
'নামজারি' এর কাছের শব্দ গুলো:  এবং, বর্তমানে, দিতে, সংশ্লিষ্ট, এখন, দুই, গভ, খতিয়ানে,
'পর্চা' এর কাছের শব্দ গুলো:  স্ল্যাশ, থাকলে, একই, না, গেলে, দেওয়া, নিন, সেটা,
'মৌজা' এর কাছের শব্দ গুলো:  পারে, জানাবেন, অপশনে, বা, আছে, অনেক, আরএস, কোনো,
Step: 30000 loss: 

In [ ]:
# Save only vector to a tsv file
# tensorflow projector only accepts this format of data
with open('kothou_keu_nei_vector.tsv', 'w') as fp:
    for i in range(VOCABULARY_SIZE):
        # i তম শব্দের জন্য এম্বেডিং
        embed = embedding[i, :]
        # convert embedding to numpy and save it to a file
        fp.write('{}\n'.format('\t'.join(map(str, embed.numpy()))))

# Save all vocabulary to a file
with open('kothou_keu_nei_metadata.tsv', 'w') as fp:
    for i in range(VOCABULARY_SIZE):
        # take word from word id
        word = id2word[i]
        # write it to a file
        fp.write('{}\n'.format(word))


## Visualize Learned Embeddings:

Load vector and metadata tsv to projector
- [https://projector.tensorflow.org/](https://projector.tensorflow.org/)
- You can also user tensorboard `projector` tab

## Resoruces
- [Efficient Estimation of Word Representations inVector Space](https://arxiv.org/pdf/1301.3781.pdf)
- [Skipgram with examples](https://www.tensorflow.org/tutorials/text/word2vec#skip-gram_and_negative_sampling)
- [Word2Vec Tensorflow 2x with low level api](https://github.com/aymericdamien/TensorFlow-Examples/blob/master/tensorflow_v2/notebooks/2_BasicModels/word2vec.ipynb)
- [Word2Vec using Embedding Layer Tensorflow 2x](https://petamind.com/word2vec-with-tensorflow-2-0-a-simple-cbow-implementation/)
- [Word2Vec](https://www.tensorflow.org/tutorials/text/word2vec)
- [Word Embeddings](https://www.tensorflow.org/tutorials/text/word_embeddings)
- [skipgram function defination](https://keras.rstudio.com/reference/skipgrams.html)